# Quickstart

This library provides functions to hone in on coincident datasets and visualize their coverage. 

The current recommended workflow is to:

1. start with a lidar dataset 
1. reduce to region with coincident maxar stereo within an acceptable temporal range
1. optionally reduce further with additional datasets such as icesat-2 and gedi altimetry

This notebook provides an example starting from USGS 3DEP LiDAR in Colorado, USA

In [ ]:
import geopandas as gpd
import coincident

print(coincident.__version__)

## Define an area of interest

Searches often start with an spatial subset, in this notebook we know we are interested in datasets in Colorado. We recommend restricting searches to areas at the 'State' scale rather than Country or Global scales

In [ ]:
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/CO/shape.geojson"
)

In [ ]:
aoi.explore(color="black", style_kwds=dict(fill=False))

```{note}
this polygon for the state of Colorado is simple and has only 4 vertices in the corners, but it's good practice good to check the number of vertices you have before searching. The simpler your search polygons are the faster your searches will be!
```

In [ ]:
print("Original number of vertices:", aoi.count_coordinates().iloc[0])
aoi = aoi.simplify(0.01)
print("Simplified number of vertices:", aoi.count_coordinates().iloc[0])

## Uniform search method

the `coincident` package provides a [search()](#coincident.search.search) method that has the same syntax regardless of which dataset you are searching. Behind the scenes, polygons intersecting your area of interest are efficiently located and returned as a geodataframe. 

For 3DEP LiDAR, we start by searching bounding boxes for each 'workunit'. Once we identify a workunit, we load a precise polygon delineating the extent of LiDAR measurements.

The search function is essentially a [pystac_client.ItemSearch](#pystac_client.ItemSearch) with an extra argument `dataset`.

In [ ]:
coincident.datasets.aliases

In [ ]:
gf = coincident.search.search(
    dataset="3dep",
    intersects=aoi,
    datetime=["2018", "2024"],
)
gf.explore(column="workunit", popup=True)

In [ ]:
# From this search we identify a specific lidar acquisition of interest
gf = gf[gf.workunit == "CO_WestCentral_2019"]
gf

Each Workunit has a unique 'Feature ID', or 'FID' that can be used to efficiently retrieve the full-resultion detailed MultiPolygon footprint from the USGS's WESM.gpkg file in AWS

In [ ]:
gf.index.values

### USGS 3DEP Lidar

Some datasets have additional functions available to load auxiliary data. For example, we can load original high resolution polygon. In addition to loading "swath" polygons to understand exact days when acquisitions were made.

```{warning}
Swath polygons are not available for all LiDAR workunits. They are more likely to exist for acquisitions after 2018.
```

In [ ]:
gf_wesm = coincident.search.wesm.load_by_fid(fids=gf.index.values)

In [ ]:
gf_wesm.explore(column="workunit", popup=True)

In [ ]:
gf_wesm[["workunit", "start_datetime", "end_datetime", "duration"]]  # duration in days

Within this 'workunit' data was collected over 29 days, in order to see when data was collected within this polygon we need to load a corresponding 'swath' polygon. `coincident` provides a helper function for this, which loads the swath polygon for a given workunit

```{note}
Swath polygons have detailed timing information for individual LiDAR flight lines composing a given 'workunit'. Not all workunits have swath polygons. They tend to be available for data collected after 2019.
```

In [ ]:
# NOTE: be patient here, reading shapefiles from s3 can be slow depending on your bandwidth
gf_swath = coincident.search.wesm.get_swath_polygons(gf.workunit.iloc[0])

In [ ]:
print("Number of swath polygons:", len(gf_swath))

In [ ]:
# Plot them, simplify first for faster plotting
gf_swath["geometry"] = gf_swath.simplify(0.01)
gf_swath.explore(column="dayofyear", cmap="plasma")

As you can see in the above figure, the actual day of observation for any point on the ground can vary in complex ways based on the flight paths taken during the LiDAR collection period.

## Maxar stereo search

Now that we understand our lidar collection, let's return to the uniform search function, and now search for Maxar stereo pairs. We are only interested in pairs in the same date range of the lidar. Note that searching by a simplified polygon (in this case back to the convex hull we started with) is recommended as most APIs have limits on the number of polygon vertices.

In [ ]:
# Simplify search polygon and add date range
pad = gpd.pd.Timedelta(days=14)
date_range = [gf.start_datetime.iloc[0] - pad, gf.end_datetime.iloc[0] + pad]
aoi = gf.geometry

gf_maxar = coincident.search.search(
    dataset="maxar",
    intersects=aoi,
    datetime=date_range,
    filter="eo:cloud_cover < 20",
)

In [ ]:
print(len(gf_maxar))
gf_maxar.head()

In [ ]:
with gpd.pd.option_context("display.max_rows", None, "display.max_columns", None):
    print(gf_maxar.iloc[0])

In [ ]:
# Group by stereo pair id (NOTE: assumes single id per mono acquisition)
gf_maxar["stereo_pair_id"] = gf_maxar.stereo_pair_identifiers.str[0]

gf_stereo = gf_maxar.dissolve(by="stereo_pair_id", as_index=False)
gf_stereo.explore(column="stereo_pair_identifiers")

In [ ]:
m = gf_wesm.explore(color="black")
gf_maxar.explore(column="dayofyear", cmap="plasma", m=m, popup=True)

### Restrict polygon footprints to area of overlap

We will make regular use of the [GeoPandas.overlay()](https://geopandas.org/en/stable/docs/reference/api/geopandas.overlay.html#geopandas-overlay) function to shrink original acquisition footprints so that they only overlap an area of interest.

In [ ]:
# Consider area of overlap
m = gf_wesm.explore(color="black")
# NOTE: don't really need wesm attributes in result, so just use geodataframe w/ geometry column
# gf_i = gf_wesm[['geometry']].overlay(gf_stereo, how='intersection')
gf_i = gf_stereo.overlay(gf_wesm[["geometry"]], how="intersection")
gf_i.explore(column="dayofyear", cmap="plasma", m=m)

### Exclude very small overlaps

Consider further filtering results by a minimum overlap area criteria, or by a more precise estimate of the number of days elapsed for the maxar stereo acquisition relative to lidar estimated from swath polygons

In [ ]:
min_area_km2 = 20
gf_i = coincident.overlaps.subset_by_minimum_area(gf_i, min_area_km2)

In [ ]:
# Look at intersection with swath polygons for exact date difference
# Since both have 'dayofyear', these cols get expanded to dayofyear_1 and dayofyear_2
stereo_pair = gf_i.iloc[[0]]
gf_dt = gf_swath.overlay(stereo_pair, how="intersection")
print("Maxar Stereo Acquisition DOY - Swath Lidar DOY:")
print(stereo_pair.stereo_pair_id.values[0])
(gf_dt.dayofyear_2 - gf_dt.dayofyear_1).describe()

## GEDI

Search for Coincident GEDI L2A.

In [ ]:
# We've refined our search polygon again, so use a new AOI
aoi = gpd.GeoSeries(gf_i.union_all().convex_hull, crs="EPSG:4326")

gf_gedi = coincident.search.search(
    dataset="gedi",
    intersects=aoi,
    datetime=date_range,
)

In [ ]:
print(len(gf_gedi))
gf_gedi.head()

In [ ]:
gf_gedi.explore()

Altimeter granules span a large geographic area! So let's again cut results down to the area of intersection.

In [ ]:
# Normalize colormap across both dataframes
vmin, vmax = gpd.pd.concat([gf_i.dayofyear, gf_gedi.dayofyear]).agg(["min", "max"])
cmap = "plasma"

# NOTE: here we just take the boundary of the union of all maxar+lidar regions to avoid many overlay geometries
union = gpd.GeoDataFrame(geometry=[gf_i.union_all()], crs="EPSG:4326")
m = gf_i.explore(column="dayofyear", cmap=cmap, vmin=vmin, vmax=vmax)
gf_gedi = union.overlay(gf_gedi, how="intersection")
gf_gedi.explore(m=m, column="dayofyear", cmap=cmap, vmin=vmin, vmax=vmax, legend=False)
m

Immediately we see some potential for close-in time acquisitions. 

```{note}
As an additional step, you might want to add another filtering step to remove tiny polygons
```

In [ ]:
# For each Stereo pair, describe number of altimeter passes and day offset
stereo_pair = gf_i.iloc[[0]]
gf_dt = stereo_pair.overlay(gf_gedi, how="intersection")
m = stereo_pair.explore(
    column="dayofyear", vmin=vmin, vmax=vmax, cmap=cmap, legend=False
)
gf_dt.explore(m=m, column="dayofyear_2", vmin=vmin, vmax=vmax, cmap=cmap)

In [ ]:
print("Maxar Stereo Acquisition DOY - GEDI DOY:")
print(stereo_pair.stereo_pair_identifiers.values[0])
(gf_dt.dayofyear_1 - gf_dt.dayofyear_2).describe()

In [ ]:
# Consider just the minimum offset for each stereo pair
# NOTE: probably a fancier way to do this using a multiindex/groupby
print("Maxar Stereo Acquisition DOY - GEDI DOY")
for i in range(len(gf_i)):
    stereo_pair = gf_i.iloc[[i]]
    gf_dt = stereo_pair.overlay(gf_gedi, how="intersection")
    min_dt = (gf_dt.dayofyear_1 - gf_dt.dayofyear_2).min()
    print(stereo_pair.stereo_pair_identifiers.values[0], min_dt)

```{note}
In this case, we have some altimeter acquisitions that were just 3 days after a maxar stereo acquisition!
```

## ICESat-2

Search for Coincident ICESat-2 Altimetry (ATL03)

In [ ]:
gf_is2 = coincident.search.search(
    dataset="icesat-2",
    intersects=aoi,
    datetime=date_range,
)
print(len(gf_is2))
gf_is2

In [ ]:
# Normalize colormap across both dataframes
vmin, vmax = gpd.pd.concat([gf_i.dayofyear, gf_is2.dayofyear]).agg(["min", "max"])
cmap = "plasma"

# NOTE: here we just take the boundary of the union of all maxar+lidar regions to avoid many overlay geometries
union = gpd.GeoDataFrame(geometry=[gf_i.union_all()], crs="EPSG:4326")
m = gf_i.explore(
    column="dayofyear", cmap=cmap, vmin=vmin, vmax=vmax, style_kwds=dict(color="black")
)
gf_is2 = union.overlay(gf_is2, how="intersection")
gf_is2.explore(m=m, column="dayofyear", cmap=cmap, vmin=vmin, vmax=vmax, legend=False)
m

In [ ]:
# For each Stereo pair, describe number of altimeter passes and day offset
stereo_pair = gf_i.iloc[[0]]
gf_dt = stereo_pair.overlay(gf_is2, how="intersection")
m = stereo_pair.explore(
    column="dayofyear",
    vmin=vmin,
    vmax=vmax,
    cmap=cmap,
    legend=False,
    style_kwds=dict(color="black"),
)
gf_dt.explore(m=m, column="dayofyear_2", vmin=vmin, vmax=vmax, cmap=cmap)

In [ ]:
print("Maxar Stereo Acquisition DOY - ICESat-2 DOY")
for i in range(len(gf_i)):
    stereo_pair = gf_i.iloc[[i]]
    gf_dt = stereo_pair.overlay(gf_is2, how="intersection")
    min_dt = (gf_dt.dayofyear_1 - gf_dt.dayofyear_2).min()
    print(stereo_pair.stereo_pair_identifiers.values[0], min_dt)

## Summary

- Any of these Maxar stereo pairs could be worth ordering. But if the goal is to have coincident acquisitions as close as possible in time '11a11283-6661-4f91-9bcb-5dab3c6a5d02-inv' seems like a good bet:
    - There is an overlapping ICESat-2 track 2 days later
    - There is an overlapping GEDI track 3 days earlier
    - 3DEP Lidar was acquired 27 to 36 days earlier

In [ ]:
# Can save this dataframe, or just note STAC ids to work with later ('102001008EC5AC00','102001008BE9BB00')
pair_id = "11a11283-6661-4f91-9bcb-5dab3c6a5d02-inv"
full_frame = gf_maxar[gf_maxar.stereo_pair_id == pair_id]
subset = gf_i[gf_i.stereo_pair_id == pair_id]

In [ ]:
# Save the original STAC metadata and polygon
full_frame.to_parquet("/tmp/maxar-mono-images.parquet")
# Also save the subsetted region for this stereo pair
subset.to_parquet("/tmp/stereo-subset.parquet")